<a href="https://colab.research.google.com/github/valmirf/processamento_imagens/blob/master/Segmentacao/Aula_05_Opencv_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Opencv - Class 5

In [ ]:
!git clone https://github.com/valmirf/processamento_imagens.git


Imports

In [ ]:
!pip install opencv-python
import cv2
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

## 1. Edge Detection

### 1.1 Canny

In [ ]:
img = cv2.imread('processamento_imagens/imagens/mangas.jpg',0)
edges = cv2.Canny(img,100,200)
#plotar
plt.figure(figsize=(6, 6), dpi= 80, facecolor='w', edgecolor='k')
plt.axis("off")
plt.imshow(edges, cmap='gray')

### 1.2 Sobel

In [ ]:
# Output dtype = cv2.CV_8U
sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
gradmag = np.sqrt(sobelx**2 + sobely**2)

fig, axes = plt.subplots(1, 2, figsize=(12, 8))
ax = axes.ravel()

ax[0].imshow(img, cmap='gray')
ax[0].set_title('Original')
ax[0].set_axis_off()

ax[1].imshow(gradmag, cmap='gray')
ax[1].set_title('Sobel')
ax[1].set_axis_off()

### 1.3 Lapracian

In [ ]:
laplacian = cv2.Laplacian(img,cv2.CV_64F)
# converting back to uint8
abs_dst = cv2.convertScaleAbs(laplacian)
fig, axes = plt.subplots(1, 2, figsize=(12, 8))
ax = axes.ravel()

ax[0].imshow(img, cmap='gray')
ax[0].set_title('Original')
ax[0].set_axis_off()

ax[1].imshow(abs_dst, cmap='gray')
ax[1].set_title('Lapraciano')
ax[1].set_axis_off()

### 1.4 Hough Transform

In [ ]:
img = cv2.imread('processamento_imagens/imagens/dave.png')
imgOut = img.copy()
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)

lines = cv2.HoughLines(edges,1,np.pi/180,200)
for i in range(len(lines)): 
    for rho,theta in lines[i]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(imgOut,(x1,y1),(x2,y2),(0,0,255),2)

fig, axes = plt.subplots(1, 2, figsize=(12, 8))
ax = axes.ravel()

ax[0].imshow(img, cmap='gray')
ax[0].set_title('Original')
ax[0].set_axis_off()

ax[1].imshow(imgOut, cmap='gray')
ax[1].set_title('Hough')
ax[1].set_axis_off()

#### 1.4.1 Probabilistic Hough Transformation

In [ ]:
imgOut2 = img.copy()
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
minLineLength = 100
maxLineGap = 10
lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
for i in range(len(lines)):
    for x1,y1,x2,y2 in lines[i]:
        cv2.line(imgOut2,(x1,y1),(x2,y2),(0,255,0),2)

fig, axes = plt.subplots(1, 2, figsize=(12, 8))
ax = axes.ravel()

ax[0].imshow(img, cmap='gray')
ax[0].set_title('Original')
ax[0].set_axis_off()

ax[1].imshow(imgOut2, cmap='gray')
ax[1].set_title('Hough')
ax[1].set_axis_off()

## 2. Segmentation

### 2.1 Limiarization

In [ ]:
img = cv2.imread('processamento_imagens/imagens/dave.png',0)
img = cv2.medianBlur(img,5)

ret,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
#th3 = cv2.threshold(img,125,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) #OTSU

titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img, th1, th2, th3]

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
ax = axes.ravel()
for i in range(4):
    ax[i].imshow(images[i], cmap='gray')
    ax[i].set_title(titles[i])
    ax[i].set_axis_off()

### 2.4 Connected Components

In [ ]:
img4 = cv2.imread('processamento_imagens/imagens/0342.jpg', 0)
img4 = cv2.threshold(img4, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary
ret, labels = cv2.connectedComponents(img4)

# Map component labels to hue val
label_hue = np.uint8(179*labels/np.max(labels))
blank_ch = 255*np.ones_like(label_hue)
labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

# cvt to BGR for display
labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

# set bg label to black
labeled_img[label_hue==0] = 0

fig, axes = plt.subplots(1, 2, figsize=(15, 8))
ax = axes.ravel()

ax[0].imshow(img4, cmap='gray')
ax[0].set_title('Original')
ax[0].set_axis_off()

ax[1].imshow(cv2.cvtColor(labeled_img, cv2.COLOR_BGR2RGB))
ax[1].set_title('Saída')
ax[1].set_axis_off()


### 2.3 Watershed

In [ ]:
img3 = cv2.imread('processamento_imagens/imagens/water_coins.jpg')
img3c = img3.copy()
gray = cv2.cvtColor(img3,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv2.watershed(img3,markers)
img3[markers == -1] = [255,0,0]

fig, axes = plt.subplots(1, 2, figsize=(12, 8))
ax = axes.ravel()

ax[0].imshow(cv2.cvtColor(img3c, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original')
ax[0].set_axis_off()

ax[1].imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
ax[1].set_title('watershed')
ax[1].set_axis_off()

### 2.2 K-Means Clustering

In [ ]:
img2 = cv2.imread('processamento_imagens/imagens/boa_viagem.jpg')
Z = img2.reshape((-1,3))

# convert to np.float32
Z = np.float32(Z)

# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 4
ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img2.shape))

fig, axes = plt.subplots(1, 2, figsize=(15, 8))
ax = axes.ravel()

ax[0].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original')
ax[0].set_axis_off()

ax[1].imshow(cv2.cvtColor(res2, cv2.COLOR_BGR2RGB))
ax[1].set_title('k-means clustering')
ax[1].set_axis_off()

### 2.3 MeanShift

#### Parameters:	
    src – The source 8-bit, 3-channel image.
    dst – The destination image of the same format and the same size as the source.
    sp – The spatial window radius.
    sr – The color window radius.
    maxLevel – Maximum level of the pyramid for the segmentation.
    termcrit – Termination criteria: when to stop meanshift iterations.

In [ ]:
meanshift = cv2.pyrMeanShiftFiltering(img2, sp=16, sr=32, maxLevel=1, 
                                      termcrit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 5, 1))
fig, axes = plt.subplots(1, 2, figsize=(15, 8))
ax = axes.ravel()

ax[0].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))#convertendo para RGB antes de plotar
ax[0].set_title('Original')
ax[0].set_axis_off()

ax[1].imshow(cv2.cvtColor(meanshift, cv2.COLOR_BGR2RGB))#convertendo para RGB antes de plotar
ax[1].set_title('MeanShift')
ax[1].set_axis_off()